In [2]:
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import getpass
import time

In [10]:
strsrtfilename = "Mass 2021 1080p.sample.srt"
output_filename = "Mass 2021 1080p.sample French.srt"
#strsrtfilename = "Mass 2021 1080p.srt"
target_language = "French"
#output_filename = "Mass 2021 1080p.French.srt"

In [4]:
OPENAI_API_KEY = getpass.getpass('Enter your OPENAI_API_KEY')

In [5]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-4o-mini")

In [6]:
with open(strsrtfilename, 'r', encoding='utf-8') as file:
    srt_text = file.read()

In [7]:
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=1000, chunk_overlap=0)
chunks = text_splitter.split_text(srt_text)

In [8]:
len(chunks)

1

In [11]:
translate_chunk_prompt_template = """You are an experienced translator to {target_language}. 
Translate the following subtitle text to {target_language} and leave the first two lines (subtitle numbers and timestamps) unchanged.
Do not add any comment to the answer, only the translated text. 
Also remove the original text in the result. 
Text: {chunk}"""
translate_chunk_prompt = PromptTemplate.from_template(translate_chunk_prompt_template)
translate_chunk_chain = translate_chunk_prompt | llm | StrOutputParser()

In [12]:
# Process chunks with a delay to respect rate limits
translations = []
batch_size = 50  # adjust batch_size if still facing rate issues

In [14]:
for i in range(0, len(chunks), batch_size):
    batch_chunks = chunks[i:i+batch_size]
    batch_translations = translate_chunk_chain.batch([{'chunk': c, 'target_language': target_language} for c in batch_chunks])
    translations.extend(batch_translations)
    print(f"Processed chunks {i+1} to {i+len(batch_chunks)}")
    time.sleep(60)  # Wait for 60 seconds between batches (adjust if necessary)

Processed chunks 1 to 1


In [15]:
# Join translations
translation = '\n\n'.join(translations)

In [16]:
translation += '\n\n'

In [17]:
# Save translation to file
with open(output_filename, 'w', encoding='utf-8') as f:
    f.write(translation)

print(f"Translation to {target_language} saved to {output_filename}")

Translation to French saved to Mass 2021 1080p.sample French.srt
